# Forensics - Create VHD Snapshot

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0, azure-cli-profile 2.1.4<br>
__OS Supported:__<br>
    -  Linux
    -  Windows
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to create Azure IaaS virtual machine snapshots for managed disks.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Prerequisite check

In [ ]:
# only run once, current version 0.1.8
#!pip install --upgrade Sentinel-Utilities

In [ ]:
import SentinelUtils
# checking Python version
check = SentinelUtils.version_management.ModuleVersionCheck()
py_check = check.validate_python('3.6.0')
if py_check.requirement_met == False:
    print('Please select Python 3.6 or Python 3.6 - AzureML at the upper right corner')
else:
    print('Please continue')

In [ ]:
# checking required packages
mods_check = check.validate_installed_modules(['azure>=4.0.0', 'azure-cli-profile>=2.1.4'])
for mod_info in mods_check:
    if mod_info.requirement_met == False:
        print('Please install {} {} at the following cell.'.format(mod_info.name, mod_info.required_version))

In [ ]:
# Please install required packages based on the check at last cell
#!pip install azure

In [ ]:
# this is needed only if you are going to use Device authentication
#!pip install azure-cli-profile

## Table of Contents

1. Authentication to Azure Resource Management 
2. Create Snapshot and Generate SAS URL

## 1. Authentication to Azure Resource Management

1. Select Authentication Methods<br>
If multi-factors authentication is enabled for your tenant, you must use either Device or Service Principal to authentication to Azure resource management.

In [ ]:
from azure.mgmt.compute import ComputeManagementClient
import SentinelAzure
# select an auth method, Device is for tenants that has MFA enabled
import ipywidgets as widgets
from IPython.display import display
auth_method = ['Device', 'Service Principal', 'User ID/Password']
selected_auth = widgets.Dropdown(options=auth_method, value=auth_method[0],description='Auth Method:')
display(selected_auth)

2. User Input and Save to Environmental store

In [ ]:
import os
# Set to True if you prefer to reset the values
reset_environment_valiables = False
env_dir = %env

if reset_environment_valiables == False and 'SUBSCRIPTION_ID' in env_dir:
    subscription_id = env_dir['SUBSCRIPTION_ID']
    print('subscription_id=' + subscription_id)
else:
    subscription_id = input('subscription_id:')
    os.environ['SUBSCRIPTION_ID'] = subscription_id
    
if reset_environment_valiables == False and 'RESOURCE_GROUP' in env_dir:
    resource_group = env_dir['RESOURCE_GROUP']
    print('resource_group=' + resource_group)
else:
    resource_group = input('resource_group:')
    os.environ['RESOURCE_GROUP'] = resource_group

3. Authentication

In [ ]:
auth = SentinelAzure.azure_aad_helper.AADHelper()
if selected_auth.value == 'Device':
    from azure.common.client_factory import get_client_from_cli_profile
    # please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
    !az login --tenant ''
    compute_client = get_client_from_cli_profile(ComputeManagementClient, subscription_id = subscription_id)
elif selected_auth.value == 'Service Principal':
    client_id=input('client_id:')
    secret=input('secret:')
    tenant_id = input('tenant_id:')
    compute_client, network_client, resource_client, storage_client = auth.authenticate('Service Principal', **{'client_id': client_id, 'secret' : secret, 'tenant_id' : tenant_id, 'subscription_id' : subscription_id})
else:
    from azure.common.credentials import UserPassCredentials
    import getpass
    uid = input('User ID:')
    pwd = getpass.getpass()
    compute_client, network_client, resource_client, storage_client = auth.authenticate('User ID Password', **{'user_id': uid, 'password' : pwd, 'subscription_id' : subscription_id})

compute = SentinelAzure.azure_compute_helper.ComputeHelper(compute_client, resource_group)

## 2. Create Snapshot and Generate SAS URL

1. Select a VM

In [ ]:
# Select a VM
import ipywidgets as widgets
from IPython.display import display
vm_names = sorted(list(vm.name for vm in list(compute_client.virtual_machines.list_all())))
selected_vm = widgets.Dropdown(options=vm_names, value=vm_names[0],description='VM:')
display(selected_vm)

In [ ]:
# User input, you may modify the default values
snapshot_name = selected_vm.value + 'snapshot'
snapshop_life_in_seconds = 3600

2. Get a Managed Disk

In [ ]:
disk_list = compute.get_vm_disk_names(selected_vm.value)
selected_disk = widgets.Dropdown(options=disk_list, value=disk_list[0],description='Disk:')
display(selected_disk)

3. Create Snapshot and Generate SAS URL

In [ ]:
snapshot = compute.create_snapshot_async(**{'snapshot_name' : snapshot_name, 'selected_disk' : selected_disk.value})
if snapshot is not None and snapshot.provisioning_state == 'Succeeded':
    sas_url = compute.generate_snapshot_sas_url_async(**{'snapshot_name' : snapshot_name, 'int_seconds' : snapshop_life_in_seconds})
    print(sas_url)